In [2]:
# -*- coding: utf-8 -*-
#https://stackoverflow.com/questions/43390466/visual-studio-community-2017-is-a-30-day-trial
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
import re, os

In [3]:
driver = webdriver.Firefox();
driver.get("http://www.zerozero.pt/player_results.php?id=159904&competicao_id=3&epoca_id=147");
try:
    driver.find_element_by_xpath("/html/body/div[1]/div/div/div[2]/button").click();
except:
    pass
driver.refresh();
#Python Calculate nth Root Function (Square, Cube, Fourth...)

# Eltiro de Ludantoj

### Retirar Equipas

In [4]:
def get_teams():
    try:
        driver.find_element_by_xpath("/html/body/div[1]/div/div/div[2]/button").click();
    except:
        pass
    
    #Site com equipas na tabela!
    driver.get('http://www.zerozero.pt/edition.php?simp=0&id_edicao=109369');
    #Refresh para retirar a publicidade!
    driver.refresh()
    
    #xpath para a tabela => ¡atencao jah deve ter mudado!
    table_ekipas="/html/body/div[7]/div[3]/div[6]/div[3]/div/div[3]/div[3]/table/tbody"
    trs=driver.find_elements_by_xpath(table_ekipas+"/tr") # => horizontal
    tds=driver.find_elements_by_xpath(table_ekipas+"/tr[1]/td") # => vertical
    print(len(trs), len(tds))
    
    #teamoj => nome de equipas, teamojn => link respectivo
    teamoj = []
    teamojn = []
    for i in range(1, len(trs)+1):
        current_team = driver.find_element_by_xpath((table_ekipas+"/tr[{}]/td[3]/a").format(str(i)))
        
        #link de cada equipa
        current_team_link = ((current_team).get_attribute('href'))
        equalSign = current_team_link.index('=')
        andSign = current_team_link.index('&')
        #extracao de ID unico de cada equipa
        #Por exemplo http://www.zerozero.pt/equipa.php?id=7&epoca_id=147 => fica ¡7!
        id_equipa = current_team_link[equalSign+1:andSign]
        
        teamoj.append(current_team) # Nome da equipa
        teamojn.append((current_team).get_attribute('href')) # Link da equipa => para colocar em ficheiro .txt
    for e in teamoj:
        if not os.path.exists('2017_2018'):
            os.makedirs('2017_2018/' + e.text)
        else:
            contrb = (str(e.text)+','+str((e).get_attribute('href')))
            print('2017_2018/' + e.text + '/equipaID.link')
            with open('2017_2018/' + e.text + '/equipaID.link', 'w+') as f:
                f.write(contrb)
    return teamoj,teamojn

### RETIRAR GIOCATORI

In [5]:
def get_players(link_equipa):
    save : list = [] #['FC Porto','Benfica','Sporting','Braga','Rio Ave','Chaves','Marítimo','Boavista','V. Guimarães','Portimonense','Tondela','Belenenses']
    driver.get(link_equipa)
    id_equipa = driver.current_url
    print(id_equipa)
    nome_equipa = driver.find_element_by_class_name('name').text
    print(nome_equipa)
    if nome_equipa in save:
        return
    equalSign = id_equipa.index('=')
    andSign = id_equipa.index('&')
    id_equipa = id_equipa[equalSign+1:andSign]
    link = ('http://www.zerozero.pt/team_players.php?id={}&epoca_stats_id=147').format(id_equipa)
    driver.get(link)
    driver.refresh()
    time.sleep(0.5)
    table_ekipas = "/html/body/div[8]/div[3]/div[6]/div[3]/div/div/table[1]/tbody/"
    trs=driver.find_elements_by_xpath(table_ekipas+"/tr") # => horizontal
    tds=driver.find_elements_by_xpath(table_ekipas+"/tr[1]/td") # => vertical
    if len(trs) == 0:
        table_ekipas = "/html/body/div[7]/div[3]/div[6]/div[3]/div/div/table[1]/tbody/"
        trs=driver.find_elements_by_xpath(table_ekipas+"/tr") # => horizontal
        tds=driver.find_elements_by_xpath(table_ekipas+"/tr[1]/td") # => vertical
        
        
    print(len(tds),len(trs))
    for j in range(1, len(trs)+1):
        jogador = driver.find_element_by_xpath((table_ekipas+"/tr[{}]/td[3]/div/div[2]/a").format(str(j)))
        contrb = (jogador.text+','+(jogador.get_attribute('href'))+'\n')
        with open('2017_2018/'+nome_equipa+'/jogadores.jfo', 'a', encoding="utf-8") as f:
            f.write(contrb)

### Juegos por Jugador

In [7]:
files = os.listdir('2017_2018')
def retirar_jogos_jogador() -> None:
    for e in range(14, 18):
        path = '2017_2018/' + files[e] + '/jogadores.jfo'; #mudar aqui
        with open(path, 'r', encoding='utf-8') as f:
            for line in f:
                jogador = line.split(',')
                if files[e] != 'FC Porto':
                    driver.get(jogador[1])
                    player_link = driver.current_url
                    equalSign = player_link.index('=')
                    id_jogador = player_link[equalSign+1:]
                    link_jogos = (('http://www.zerozero.pt/player_results.php?id={}&competicao_id=3&epoca_id=147').format(id_jogador))
                    driver.get(link_jogos)
                    trs = driver.find_elements_by_css_selector(".box_table table:nth-child(1) tbody:nth-child(2) tr")
                    tds = driver.find_elements_by_css_selector(".box_table table:nth-child(1) tbody:nth-child(2) tr:nth-child(1) td")
                    print(len(trs), len(tds))
                    for j in range(1, len(trs)+1):
                        vitderr = driver.find_element_by_xpath(('//table/tbody/tr[{}]/td[@class="form"]').format(j)).text
                        current_game = driver.find_element_by_xpath(('//table/tbody/tr[{}]/td[7]/a').format(j)).get_attribute('href')
                        entrou_saiu = driver.find_element_by_css_selector(('.box_table table:nth-child(1) tbody:nth-child(2) tr:nth-child({}) td:nth-child(9)').format(j))
                        tempo_em_jogo = driver.find_element_by_css_selector(('.box_table table:nth-child(1) tbody:nth-child(2) tr:nth-child({}) td:nth-child(8)').format(j))
                        minutos_totais = driver.find_elements_by_xpath("//div[@class='box']/div[@id='facts']/div[@class='curiosity']/div[@class='facts_graphs']/div[@class='graph_games']/div[@style='width:109px;float:left;display:block;']/div[@class='value']")
                        try:
                            minutos_totais = minutos_totais[1].text.replace('\nMinutos', '')
                        except:
                            minutos_totais = 0
                        print(vitderr, current_game, entrou_saiu.text, tempo_em_jogo.text, minutos_totais)
                        contrb = (('{},{},{},{},{}\n').format(vitderr, current_game, entrou_saiu.text, tempo_em_jogo.text, minutos_totais));
                        print(contrb)
                        path_folder = (('2017_2018/{}/{}.jg').format(files[e], jogador[0]))
                        with open(path_folder, 'a') as f:
                            f.write(contrb)
                else:
                    pass

In [ ]:
retirar_jogos_jogador()

# Recenzja Gry

In [9]:
def table_results():
    fora = driver.find_element_by_xpath('//div[@class="away"]//div[@class="team"]').text
    casa = driver.find_element_by_xpath('//div[@class="home"]//div[@class="team"]').text
    equipa_fora_golos = driver.find_elements_by_xpath('//div[@class="away"]//div[@class="scorers"]//span[@class="time"]')
    equipa_casa_golos = driver.find_elements_by_xpath('//div[@class="home"]//div[@class="scorers"]//span[@class="time"]')
    golls_fora : list = []
    golls_casa : list = []
    for g in equipa_fora_golos:
        gl : list = g.text.replace(' (g.p.)', '').replace(' (p.b.)', '').split(' ')
        e : int
        for e in range(len(gl)):
            golls_fora.append(eval(gl[e]))
        
    for g in equipa_casa_golos:
        gl : list = g.text.replace(' (g.p.)', '').replace(' (p.b.)', '').split(' ')
        e : int
        for e in range(len(gl)):
            golls_casa.append(eval(gl[e]))
                
    
    return casa, golls_casa, fora, golls_fora

In [10]:
def goal_comparison(files, casa, goloj_casa, fora, goloj_fora, minutos_jogados):
    gm : int = 0
    gs : int = 0
    if files == casa:
        print('casa')
        for g in range(len(goloj_casa)):
            if (goloj_casa[g] >= minutos_jogados[0]) and (goloj_casa[g] <= minutos_jogados[1]):
                gm += 1
        for g in range(len(goloj_fora)):
            if (goloj_fora[g] >= minutos_jogados[0]) and (goloj_fora[g] <= minutos_jogados[1]):
                gs += 1

    elif files == fora:
        print('fora')
        for g in range(len(goloj_casa)):
            if (goloj_casa[g] >= minutos_jogados[0]) and (goloj_casa[g] <= minutos_jogados[1]):
                gs += 1
        for g in range(len(goloj_fora)):
            if (goloj_fora[g] >= minutos_jogados[0]) and (goloj_fora[g] <= minutos_jogados[1]):
                gm += 1
                
    return gm, gs

### L'Algorithme 

In [ ]:
def comparacion():
    # adicionar um to_exclude!
    to_exclude = ['equipaID.link', 'jogadores.jfo']
    files = os.listdir('2017_2018')
    for i in range(1):
        i = 1
        n_jogadors : int = 0
        path : str = '2017_2018/{}/jogadores.jfo'.format(files[i])
        with open(path, 'r', encoding='utf-8') as f:
            line : str
            for line in f:
                n_jogadors += 1
        for j in range(n_jogadors):
            files = os.listdir('2017_2018'); #files = Folders with team's names!
            jogadores = os.listdir('2017_2018/{}'.format(files[i]));
            if files[i] in to_exclude:
                pass
            else:
                path = ('2017_2018/{}/{}'.format(files[i],jogadores[j]))
                save = os.listdir(('2017_2018/{}/stats/').format(files[i]))
                print(save)
                if jogadores[j] not in save:
                    with open(path, 'r', encoding='utf-8') as f:
                        for line in f:
                            dados_jogo = line.split(',')
                            if len(dados_jogo) > 3:        
                                if dados_jogo[3] == 'NU':
                                    pass
                                else:
                                    driver.get(dados_jogo[1])
                                    casa, goloj_casa, fora, goloj_fora = table_results()
                                    print(casa, goloj_casa, fora, goloj_fora)
                                    minutos_jogados = 0
                                    if dados_jogo[2] == '-':
                                        minutos_jogados = [0, int(dados_jogo[3])]
                                    elif dados_jogo[2][0] == '8':
                                        get_m = dados_jogo[2].replace('\'', '').split(' ')
                                        minutos_jogados = [0, int(get_m[1])]
                                    elif dados_jogo[2][0] == '7':
                                        get_m = dados_jogo[2].replace('\'', '').split(' ')
                                        tt = int(dados_jogo[3]) + int(get_m[1])
                                        minutos_jogados = [int(get_m[1]), tt]

                                    gm : int = 0
                                    gs : int = 0
                                    print(minutos_jogados)
                                    gm, gs = goal_comparison(files[i], casa, goloj_casa, fora, goloj_fora, minutos_jogados)
                                    tempo : int = (minutos_jogados[1] - minutos_jogados[0])
                                    path_sts : str = ('2017_2018/{}/stats/{}.sts'.format(files[i],jogadores[j][:-3]))
                                    t_write : str = '{},{},{}\n'.format(gm, tempo, gs)
                                    print(t_write)
                                    with open(path_sts , 'a', encoding='utf-8') as f:
                                        f.write(t_write)
                else:
                    print(jogadores[j], "jah esta na db")

In [ ]:
comparacion()

Benfica [51] Moreirense []
[70, 90]
casa
0,20,0

Sporting [] Benfica []
[0, 90]
fora
0,90,0

Benfica [12, 94] Tondela [31, 39, 81]
[0, 90]
casa
1,90,3

Estoril Praia [63] Benfica [10, 92]
[90, 91]
fora
0,1,0

Benfica [] FC Porto [90]
[74, 90]
casa
0,16,1



# From now on, there'll only be ¡TESTS!